# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
R. Hoppe  ->  R. Hoppe  |  ['R. Hoppe']
J. Li  ->  J. Li  |  ['J. Li']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
F. Xu  ->  F. Xu  |  ['F. Xu']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']


Arxiv has 65 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.03787


extracting tarball to tmp_2511.03787...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.04254


extracting tarball to tmp_2511.04254...

...

extracting tarball to tmp_2511.04459...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.04504


extracting tarball to tmp_2511.04504...

 done.


/tmp/ipykernel_3455/2822249172.py:52: LatexWarning: 2511.04504 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.04280-b31b1b.svg)](https://arxiv.org/abs/2511.04280) | **The Initial mass function of field stars with mass $\leq$ 1 $M_{\odot}$ varies with metallicity**  |
|| D. Qiu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-07*|
|*Comments*| *12 pages, 13 figures*|
|**Abstract**|            We investigated a volume-limited sample of LAMOST main-sequence stars with masses from 0.25 to 1 $M_{\odot}$ and distances of 150-350 pc to explore how the stellar initial mass function (IMF) varies with metallicity. We corrected the spectroscopic selection function by comparing the stellar number densities with the photometric ones at the same colour and magnitude. From these corrected number density distributions, we derived IMFs for each metallicity sub-samples. Fitting a broken power-law function in each IMF with a fixed break point at 0.525 $M_{\odot}$, we found the power-law indices increase with [Fe/H] for both mass regimes: $\alpha_1$ (mass $\leq$ 0.525 $M_{\odot}$) rises from 0.54 $\pm$ 0.21 to 1.40 $\pm$ 0.07 and $\alpha_2$ (mass>0.525 $M_{\odot}$) grows from 1.40 $\pm$ 0.16 to 1.86 $\pm$ 0.04 as [Fe/H] varies from -1 to +0.5 dex. It demonstrates that low-mass stars make up a larger fraction in metal-rich environments than in metal-poor ones. We performed simulations to assess the impact of unresolved binaries on the IMF power-law indices. After correction, the binary-adjusted $\alpha$ values retained a similar metallicity-dependent trend. Furthermore, by examining the IMF of the aggregate sample, we found the corrected indices ($\alpha_{\rm{1,corr}} = 1.48 \pm 0.03$ , $\alpha_{\rm{2,corr}} = 2.17 \pm 0.03$) are consistent with Kroupa's IMF values ($\alpha_1 = 1.3 \pm 0.5$ and $\alpha_2 = 2.3 \pm 0.3$). Finally, we verified the robustness of our results by testing different break points and mass bin sizes, confirming that the IMF's dependence on [Fe/H] remains consistent.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.03787-b31b1b.svg)](https://arxiv.org/abs/2511.03787) | **Probing the dawn of galaxies: star formation and feedback in the JWST era through the GAEA model**  |
|| S. Cantarella, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-11-07*|
|*Comments*| *21 pages, 16 figures, submitted to A&A*|
|**Abstract**|            The James Webb Space Telecope (JWST) opened a new window for the study of the highest redshift ($z>7$) Universe. This work presents a theoretical investigation of the very-high redshift Universe using the state-of-the-art GALaxy Evolution and Assembly (GAEA) model, run on merger trees from the Planck-Millennium $N$-body simulation. We show that GAEA successfully reproduces a wide range of high-$z$ observational estimates including: the galaxy stellar mass function up to $z\sim13$ and the total (galaxies and AGN) UV luminosity function (LF) up to $z\sim10$. We find that the AGN UV emission represents an important contribution at the bright end of the UVLF up to $z\sim8$, but it is negligible at higher redshift. Our model reproduces well the observed mass-metallicity relation at $z\leq4$, while it slightly overestimates the normalization of the relation at earlier cosmic epochs. At $z\geq11$, current UVLF estimates are at least one order of magnitude larger than model predictions. We investigate the impact of different physical mechanisms, such as an enhanced star formation efficiency coupled with a reduced stellar feedback or a negligible stellar feedback at $z>10$. In the framework of our model, both the galaxy stellar mass and UV luminosity functions at $z\geq10$ can be explained by assuming feedback-free starbursts in high-density molecular clouds. However, we show that this model variant leads to a slight increase of the normalization of the $z\geq10$ mass-metallicity relation, strengthening the tension with available data. A model with negligible stellar feedback at $z>10$ also predicts larger numbers of massive and bright galaxies aligning well with observations, but it also overestimates the metallicity of the interstellar medium. We show that these model variants can in principle be discriminated using the relation between the star formation rate and galaxy stellar mass.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.04400-b31b1b.svg)](https://arxiv.org/abs/2511.04400) | **Artificial Precision Polarization Array: Sensitivity for the axion-like dark matter with clock satellites**  |
|| <mark>H. Jiang</mark>, B. Xu, Y.-L. Zhang |
|*Appeared on*| *2025-11-07*|
|*Comments*| *11 pages, 9 figures*|
|**Abstract**|            The approaches to searching for axion-like signals based on pulsars include observations with pulsar timing arrays (PTAs) and pulsar polarization arrays (PPAs). However, these methods are limited by observational uncertainties arising from multiple unknown and periodic physical effects, which substantially complicate subsequent data analysis. To mitigate these issues and improve data fidelity, we propose the Artificial Pulsar Polarization Arrays (APPA): a satellite network comprising multiple pulsed signal transmitters and a dedicated receiver satellite. In order to constrain the axion-photon coupling parameter $g_{a\gamma}$, we generate simulated observations using Monte Carlo methods to investigate APPA's sensitivity via two complementary approaches: Bayesian analysis and frequentist analysis. Simulations indicate that for axion mass $m_{a}\sim\mathcal{O}\big(10^{-22}-10^{-19}\big)$ eV, APPA yields a better upper limit on $g_{a\gamma}$ (at the 95\% confidence level) than conventional ground-based observations and achieves better detection sensitivity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.04459-b31b1b.svg)](https://arxiv.org/abs/2511.04459) | **Study the nature of dynamical dark energy by measuring the CMB polarization rotation angle**  |
|| H. Zhai, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-07*|
|*Comments*| *16 pages,10 figures*|
|**Abstract**|            Recent results from the Dark Energy Spectroscopic Instrument (DESI) support the dynamical dark energy. Intriguingly, the data favor a transition of the dark energy equation of state across $w=-1$, a hallmark of the Quintom scenario. In this paper, we consider a different approach to the dynamical nature of dark energy by investigating its interaction with ordinary matters, specifically the Chern-Simons (CS) interaction with photons. In cosmology, this interaction rotates the polarized plane of the cosmic microwave background (CMB) photons, which induces non-zero polarized TB and EB power spectra. We forecast this measurement with the Ali CMB Polarization Telescope (AliCPT) experiment. We take the best-fit value of the isotropic rotation angle from Planck data as our fiducial input. We project that 11 module-year (modyr) of observations will yield an improved detection sensitivity with a significance $\sim 5\sigma$, given a calibration precision of $0.1^\circ$ in the polarization angle. We also forecast AliCPT's sensitivity to the amplitude of a scale invariant spectrum of the anisotropic polarization rotation field. With $50$~modyr of observations, the large-aperture configuration is expected to reach $\sigma_{A_{\mathrm{CB}}}\sim10^{-2}$, offering a sixfold improvement over the small-aperture design and enabling competitive tests of spatial fluctuations in the dark energy field.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.04254-b31b1b.svg)](https://arxiv.org/abs/2511.04254) | **3D Non-LTE radiation transfer: theory and applications to stars, exoplanets, and kilonovae**  |
|| <mark>M. Bergemann</mark>, <mark>R. Hoppe</mark> |
|*Appeared on*| *2025-11-07*|
|*Comments*| *Invited review for Living Reviews in Computational Astrophysics (LRCA), under review, 143 pages, constructive comments and suggestions welcome*|
|**Abstract**|            Most of the physical information about astrophysical objects is obtained via the analysis of their electromagnetic spectra. Observed data coupled with radiation transfer models in physical conditions representative of stars, planets, kilonovae, and ISM, yield constrains on their physical structure, gas flow dynamics at the surface, mass loss, and detailed chemical composition of the systems. All these core astrophysical parameters are just as reliable as the physical quality of the models that are employed for simulations of radiation transfer. Recent advances in multi-D transfer modeling with Non-Local Thermodynamic Equilibrium (NLTE) in inhomogeneous time-dependent systems revealed systematic shortcomings of canonical models. Owing to major complexities of solving coupled multi-frequency RT equations in 3D geometry, a number of approximations have been introduced. This review presents an overview of the physical problem, standard solutions, and recent methodological advances. We also provide an overview of main results in the area of 3D NLTE radiation transfer and its applications to modeling diverse astrophysical environments, including FGKM type- and OBA-type stars, multi-epoch spectra of kilonovae, and atmospheres of rocky and gaseous exoplanets.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.04504-b31b1b.svg)](https://arxiv.org/abs/2511.04504) | **The ALMA-ATOMS-QUARKS survey: Resolving a chemically rich massive protostellar outflow**  |
|| J.-H. Zou, et al. -- incl., <mark>F. Xu</mark>, <mark>P. Garcia</mark> |
|*Appeared on*| *2025-11-07*|
|*Comments*| *Accepted by ApJ on 4 November 2025*|
|**Abstract**|            We present a comprehensive study on the physical and chemical structures of a chemically rich bipolar outflow in a high-mass star forming region IRAS 16272$-$4837 (SDC335), utilizing high-resolution spectral line data at 1.3 mm and 3 mm dual-bands from the ALMA ATOMS and QUARKS surveys. The high-velocity jet is enveloped by a lower-velocity outflow cavity, containing bright knots that show enhanced molecular intensities and elevated excitation temperatures. Along the outflow, we have identified 35 transitions from 22 molecular species. By analyzing the spatial distribution and kinematics of these molecular lines, we find that the molecular inventory in the outflow is regulated by three processes: (i) direct entrainment from the natal molecular core by the outflow; (ii) shock-induced release of molecules or atoms from dust grains; and (iii) thermal desorption and gas-phase reactions driven by shock heating. These results confirm that outflows are not only dynamical structures but also active chemical factories, where entrainment, shocks, and thermal processing jointly enrich the molecular content. Our findings confirmed that outflow chemistry has multi-origin nature, and provide critical insights into chemical evolution during high-mass star formation.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.04280/./Fig/dndm_mass.png', 'tmp_2511.04280/./Fig/alpha_feh.png', 'tmp_2511.04280/./Fig/teff_feh.png']
copying  tmp_2511.04280/./Fig/dndm_mass.png to _build/html/
copying  tmp_2511.04280/./Fig/alpha_feh.png to _build/html/
copying  tmp_2511.04280/./Fig/teff_feh.png to _build/html/
exported in  _build/html/2511.04280.md
    + _build/html/tmp_2511.04280/./Fig/dndm_mass.png
    + _build/html/tmp_2511.04280/./Fig/alpha_feh.png
    + _build/html/tmp_2511.04280/./Fig/teff_feh.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feh}{\rm[Fe/H]}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\M}{M_{\odot}}$
$\newcommand{\orcid}[1]{$
$  \href{https://orcid.org/#1}{\textcolor{orcidgreen}{\faOrcid}}$
$}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The Initial mass function of field stars with mass $\leq$1 $M_{\odot}$ varies with metallicity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.04280-b31b1b.svg)](https://arxiv.org/abs/2511.04280)<mark>Appeared on: 2025-11-07</mark> -  _12 pages, 13 figures_

</div>
<div id="authors">

D. Qiu, et al. -- incl., <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** We investigated a volume-limited sample of LAMOST main-sequence stars with masses from 0.25 to 1 $\M$ and distances of 150-350 pc to explore how the stellar initial mass function (IMF) varies with metelliaicty.We corrected the spectroscopic selection function by comparing the stellar number densities with the photometric ones at the same colour and magnitude. From these corrected number density distributions, we derived IMFs for each metallicity sub-samples. Fitting a broken power-law function in each IMF with a fixed break point at 0.525 $\M$ , we found the power-law indices increase with [ Fe/H ] for both mass regimes: $\alpha_1$ (mass $\leq$ 0.525 $\M$ ) rises from 0.54 $\pm$ 0.21 to 1.40 $\pm$ 0.07 and $\alpha_2$ (mass $>$ 0.525 $\M$ ) grows from 1.40 $\pm$ 0.16 to 1.86 $\pm$ 0.04as [ Fe/H ] varies from -1 to +0.5 dex. It demonstratesthat low-mass stars make up a larger fraction in metal-rich environments than in metal-poor ones. We performed simulations to assess the impact of unresolved binaries on the IMF power-law indices.After correction, the binary-adjusted $\alpha$ values retained a similar metallicity-dependent trend.Furthermore, by examining the IMF of the aggregatesample,we found the corrected indices ( $\alpha_{\rm{1,corr}} = 1.48 \pm 0.03$ , $\alpha_{\rm{2,corr}} = 2.17 \pm 0.03$ ) are consistent with Kroupa's IMF values ( $\alpha_1 = 1.3 \pm 0.5$ and $\alpha_2 = 2.3 \pm 0.3$ ). Finally, we verified the robustness of our results by testing different break points and mass bin sizes, confirming that the IMF’s dependence on [ Fe/H ] remains consistent.

</div>

<div id="div_fig1">

<img src="tmp_2511.04280/./Fig/dndm_mass.png" alt="Fig11" width="100%"/>

**Figure 11. -** Both axes are shown on logarithmic scales. The solid lines, transitioning from yellow to dark purple, represent the initial mass function of stars with metallicities ranging from –1.0 to +0.5 dex, the black solid line denotes the IMF derived from the full, unbinned sample. All the solid lines are computed with a mass bin size of 0.05$\M$. The dotted lines show the corresponding IMFs calculated with a coarser mass bin size of 0.1 $\M$. A vertical pink solid line marks the location of 0.525 $\M$, corresponding to the mass bin [0.50,0.55) $\M$(highlighted in pink). Two vertical pink dotted lines at 0.475 $\M$  and 0.575$\M$  indicate the adjacent bins, [0.45,0.50) and [0.55,0.60)$\M$, respectively. (*fig:dndm*)

</div>
<div id="div_fig2">

<img src="tmp_2511.04280/./Fig/alpha_feh.png" alt="Fig3" width="100%"/>

**Figure 3. -** The red and blue lines represent the IMF power-law indices as a function of $\feh$ for stars with mass $\leq$ 0.525 $\M$($\alpha_1$) and > 0.525 $\M$($\alpha_2$), respectively. The black line displays the result of Li23 as a reference. The yellow dashed line marks the Kroupa's slope for stars with a mass range of [0.08, 0.5) $M_{\odot}$($\rm \alpha_{1,Kroupa}=1.3$), with the surrounding yellow shading indicating its quoted uncertainty ($\pm$0.5).
Likewise, the purple line and shading show the slop value and uncertainty for mass $\geq$ 0.5 $\M$($\rm \alpha_{2,Kroupa} =2.3\pm 0.3$).
  (*fig:alpha_feh*)

</div>
<div id="div_fig3">

<img src="tmp_2511.04280/./Fig/teff_feh.png" alt="Fig1" width="100%"/>

**Figure 1. -** The metallicity versus the effective temperature of 1308 LAMOST M dwarfs. The metallicities, calibrated with Equation (\ref{eq:deltafeh}), are inherited from the F, G, or K dwarf companions, whereas the $\teff$ are taken from the LASPM pipeline. (*fig:teff_feh*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.04280"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

120  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

19  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

10  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
